In [1]:
import json
import requests
import pandas as pd
import numpy as np
import random
import sqlalchemy
#from pandas.io.json import json_normalize

In [2]:
df = []

In [3]:
with open('Redtube_list.txt', 'r') as filehandle:
    df = json.load(filehandle)

In [4]:
#Table1 = json_normalize(df)

In [5]:
# for i in random.sample(range(1500000, 1500000), 5000):
#     url = f'https://api.redtube.com/?data=redtube.Videos.getVideoById&video_id={i}&output=json&thumbsize=all'
#     response = requests.get(url)
#     print(response.status_code)
#     if response.status_code == 200:
#         data = response.json()
#         df.append(data)
#         print(f'Finished df temp for number {i}')   
# print('Completed')

In [8]:
video = []
for i in df:
    for key, value in i.items():
        if key == 'video':
            video.append(value)

In [9]:
tags = []
for l in video:
    for k,v in l.items():
        if k == 'tags':
            tags.append(v)

In [10]:
views = np.array([])
rating = np.array([])
titles = np.array([])
#duration = np.array([])
#number_of_rates = np.array([])
#video_id = np.array([])

for dict1 in video:
    for key1, value1 in dict1.items():
        if key1 == 'views':
            views = np.append(views, np.array([value1]))
        elif key1 == 'rating':
            rating = np.append(rating, np.array([value1]))
        elif key1 == 'title':
            titles = np.append(titles, np.array([value1]))
#         elif key1 == 'video_id':
#             video_id = np.append(video_id, np.array([value1]))
#         elif key1 == 'duration':
#             duration = np.append(duration, np.array([value1]))
#         elif key1 == 'ratings':
#             number_of_rates = np.append(number_of_rates, np.array([value1]))            

In [11]:
categories = pd.DataFrame(tags)

In [12]:
categories = categories.drop(categories.columns[[3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]], axis=1)

In [13]:
categories.columns = ['Category_1', 'Category_2', 'Category_3']

In [14]:
arrays = {'Title': titles, 'Number_of_Views': views, 'Rating': rating}
table = pd.DataFrame(data=arrays)

In [15]:
full_table = pd.merge(table, categories, left_index=True, right_index=True)

In [16]:
full_table['Website'] = 'Redtube'

In [17]:
full_table = full_table.astype({'Title': str, 'Number_of_Views': int, 'Rating': float, 'Category_1': str, 'Category_2': str, 'Category_3': str, 'Website': str})

In [18]:
full_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51901 entries, 0 to 51900
Data columns (total 7 columns):
Title              51901 non-null object
Number_of_Views    51901 non-null int32
Rating             51901 non-null float64
Category_1         51901 non-null object
Category_2         51901 non-null object
Category_3         51901 non-null object
Website            51901 non-null object
dtypes: float64(1), int32(1), object(5)
memory usage: 2.6+ MB


In [19]:
full_table.head()

,Title,Number_of_Views,Rating,Category_1,Category_2,Category_3,Website
0,Glamour girl pleasuring herself,215864,88.1962,Amateur,Blonde,Caucasian,Redtube
1,Sexy artist seducing and bodypainting,107805,87.3641,Brunette,Glamour,Latin,Redtube
2,Sexy teen webcam striptease,627376,88.1806,Amateur,Black-haired,Latin,Redtube
3,Sweet girl deepthroating,152832,90.7239,Bikini,Blowjob,Caucasian,Redtube
4,Cybersex mistake,117170,88.1584,Caucasian,Couple,Funny,Redtube


In [20]:
full_table.to_csv(r'C:\\Users\\user\\Downloads\\Data Analytics\\Directory\\Project-Week-3-Data-Thieves\\your-project\\Redtube_table.csv')

In [21]:
driver = 'mysql+pymysql'
user = 'root'
password = 'Ironhack10'
ip = '35.204.21.134'
database = 'porn_analysis'

conn_string = f'{driver}://{user}:{password}@{ip}/{database}'
    
conn = sqlalchemy.create_engine(conn_string)

In [34]:
full_table.to_sql('main', conn, if_exists = 'append')

In [49]:
# open output file for writing
with open('Ali_list.txt', 'w') as filehandle:
    json.dump(df, filehandle)

In [50]:
#https://api.redtube.com/docs